In [ ]:
import pandas as pd

In [ ]:
data_filepath = "../data/January-2024.csv"
df = pd.read_csv(data_filepath)
month_year = data_filepath.split("/")[-1].split(".")[0]

In [ ]:
month_year

In [ ]:
# Fix wrong postage

def convert_negative_to_positive(value):
    if value < 0:
        return abs(value)
    return value

# Applying the function to the 'postage' column
df['postage'] = df['postage'].apply(convert_negative_to_positive)

This notebook makes some really important business reports that are really important to us

In [ ]:
# Calculate sales by area

salesdf = df.groupby("State")["item_count"].agg({"sum", "min", "max", "count"}).reset_index()
salesdf.head()

In [ ]:
age_intervals = [0,18, 30, 50, 70, 120]

In [ ]:
def put_in_age_group(age:int)-> str:
    if age < 18:
        group = "0-18"
    elif 18 <= age < 30:
        group = "18-30"
    elif 30 <= age < 50:
        group = "30-50"
    elif 50 <= age < 70:
        group = "50-70"
    else:
        group = "over 70"
    
    return group

In [ ]:
df['group'] = df['Age'].apply(put_in_age_group)
df

In [ ]:
sales_by_age_group_df = df.groupby("group")["item_count"].agg({"mean","median", "count"}).reset_index()
sales_by_age_group_df

In [ ]:
data = {"sales_by_region": salesdf,"sales_by_age":sales_by_age_group_df}

In [ ]:
output_path = f"/Users/jayminmistry/Documents/quality-data-pipelines/data/super_critical_business-report-{month_year}.xlsx"
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
  #  data = [sales_by_age_group_df, salesdf]
    for sheetname, frame in data.items():
        frame.to_excel(writer, sheet_name = sheetname, startrow=2, index=False, header=False)
        header_format = writer.book.add_format(
            {"bold": True, "text_wrap": True, "valign": "top", "border": 1}
        )
        for col_num, value in enumerate(frame.columns.values):
            writer.sheets[sheetname].write(1, col_num, value, header_format)

        # apply autofiltering and freeze first row and column
        writer.sheets[sheetname].freeze_panes(2, 1)